<a href="https://colab.research.google.com/github/yfb2022/alphachodc/blob/main/NEWS_SUMMARIZER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Investing.com 크롤링

In [ ]:
import sys

# 설치 중 엔터 필수 Press[ENTER]
!sudo add-apt-repository ppa:saiarcot895/chromium-beta
!sudo apt remove chromium-browser
!sudo snap remove chromium
!sudo apt install chromium-browser

!pip3 install selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

 This PPA contains the latest Chromium Beta builds, with hardware video decoding enabled (hidden behind a flag), and support for Widevine (needed for viewing many DRM-protected videos) enabled.

== Hardware Video Decoding ==

To enable hardware video decoding, start Chromium with the --enable-features=VaapiVideoDecoder argument. To make this persistent, create a file at /etc/chromium-browser/customizations/92-vaapi-hardware-decoding with the following contents:

CHROMIUM_FLAGS="${CHROMIUM_FLAGS} --enable-features=VaapiVideoDecoder"

See also https://wiki.archlinux.org/title/Chromium#Hardware_video_acceleration for more information on VAAPI video decoding support.

=== Widevine Support ===

The packages in this PPA have support for Widevine inside Chromium enabled. However, you still need to copy some files from Chrome into Chromium for you to use Netflix (or other websites using Encrypted Media Extensions) in Chromium.

1. Download and install Chrome (or extract the necessary files, if

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
import time

options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

webdriver_service = Service('/usr/bin/chromedriver')
driver = webdriver.Chrome(service = webdriver_service, options=options)

url = 'https://www.investing.com/news/most-popular-news'

driver.get(url)
article_title = driver.find_elements(By.CSS_SELECTOR, '#leftColumn > div.largeTitle > article > div.textDiv > a')
article_source = driver.find_elements(By.CSS_SELECTOR, '#leftColumn > div.largeTitle > article > div.textDiv > span')
article_links = driver.find_element(By.CLASS_NAME, 'largeTitle').find_elements(By.CLASS_NAME,'js-article-item')

titles = [x.text for x in article_title]
sources = [x.text for x in article_source]
links = []
times = []
stocks = []
contents = []

for i in article_links:
    links.append(i.find_element(By.CSS_SELECTOR, 'a').get_attribute('href'))

for i in links:
    driver.get(i)
    article_contents = driver.find_elements(By.CSS_SELECTOR, '#leftColumn > div.WYSIWYG.articlePage > p')
    try:
      article_time = driver.find_element(By.CSS_SELECTOR, '#leftColumn > div:nth-child(6) > span')
    except:
      article_time = ' '
    try:
      article_stocks = driver.find_element(By.CSS_SELECTOR, '#leftColumn > div.WYSIWYG.articlePage > div.relatedInstrumentsWrapper > div > div.slider')
    except:
      article_stocks = ' '
    content = [x.text for x in article_contents]
    content_txt = ' '.join(content)
    contents.append(content_txt)
    if isinstance(article_time, str) == False:
      times.append(article_time.text)
    else:
      times.append(article_time)
    if isinstance(article_stocks, str) == False:
      stocks.append(article_stocks.text)
    else:
      stocks.append(article_stocks)

driver.close()

In [ ]:
import pandas as pd

# 열 텍스트 전처리
for i in sources:
    if i == '':
        sources.remove(i)

for i in range(len(sources)):
  sources[i] = sources[i].split('-')[0].strip()

for i in range(len(times)):
  times[i] = times[i].replace('Published','').replace('ET','')

for i in range(len(stocks)):
  stocks[i] = stocks[i].replace('\n', ':').replace('%:', '%, ')

df = pd.DataFrame({'title':titles, 'time':times, 'stocks': stocks, 'content':contents, 'source': sources, 'link':links})
removal_list = []

for i in range(len(df)):
    if df['content'].iloc[i] == '':
        removal_list.append(i)

df.drop(removal_list, axis=0, inplace=True)
df.reset_index(inplace=True, drop=True)
df

,title,time,stocks,content,source,link
0,Germany refusing Intel's additional demand for...,"Jun 11, 2023 12:29AM",INTC:-1.51%,(Reuters) - Germany's Finance Minister Christi...,By Reuters,https://www.investing.com/news/stock-market-ne...
1,"Zelenskiy says counteroffensive actions ""takin...","Jun 10, 2023 09:20PM",,By Tom Balmforth KYIV (Reuters) -President Vol...,By Reuters,https://www.investing.com/news/world-news/zele...
2,Energy & precious metals - weekly review and o...,"Jun 11, 2023 05:03AM","XAU/USD:-0.24%, Gold:-0.14%, LCO:-1.21%, CL:-1...",Investing.com -- “Patience” - it’s a commodity...,By Investing.com,https://www.investing.com/news/commodities-new...
3,Bidens offer 'joy' at White House Pride event ...,"Jun 10, 2023 04:15PM",,By Andrea Shalal WASHINGTON (Reuters) - U.S. P...,By Reuters,https://www.investing.com/news/world-news/bide...
4,Four Colombian children found alive in jungle ...,"Jun 09, 2023 08:22PM",,By Luis Jaime Acosta BOGOTA (Reuters) - Four c...,By Reuters,https://www.investing.com/news/world-news/four...
5,Why is the crypto market down today?,"Jun 10, 2023 08:19AM","BTC/USD:+0.09%, USDT/USD:-0.03%, BTC/USD:+0.11...",The cryptocurrency market fell 7% below $1 tri...,By Cointelegraph,https://www.investing.com/news/cryptocurrency-...
6,Twitter is refusing to pay its Google Cloud bi...,"Jun 10, 2023 07:22PM","GOOGL:+0.07%, AMZN:-0.66%, GOOG:+0.16%",(Reuters) - Twitter has refused to pay its Goo...,By Reuters,https://www.investing.com/news/stock-market-ne...
7,U.S. gives Havana embassy a facelift after yea...,"Jun 10, 2023 07:04AM",,By Dave Sherwood HAVANA (Reuters) -When the U....,By Reuters,https://www.investing.com/news/world-news/us-g...
8,Trump magnifies attacks on Justice Department ...,"Jun 10, 2023 04:18PM",,By Nathan Layne (Reuters) -Former President Do...,By Reuters,https://www.investing.com/news/world-news/trum...
9,BioNTech faces first German lawsuit over alleg...,"Jun 11, 2023 05:56AM",PFE:-0.31%,By Ludwig Burger and Patricia Weiss HAMBURG (R...,By Reuters,https://www.investing.com/news/stock-market-ne...


In [ ]:
df.to_csv('datafile.csv')

# 텍스트 전처리

In [ ]:
!pip install nltk
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download('averaged_perceptron_tagger')
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize

wnl = WordNetLemmatizer()

# 기사 앞 필요없는 부분 제거
def text_preprocessing(doc):
    sent_list = nltk.sent_tokenize(doc)

    match = re.search(r"(\) -| -- | - )", sent_list[0])
    if match:
        return doc[match.end():].strip()
    else:
        return doc

def pos_tagger(nltk_tag):
    if nltk_tag.startswith('J'):
        return wordnet.ADJ
    elif nltk_tag.startswith('V'):
        return wordnet.VERB
    elif nltk_tag.startswith('N'):
        return wordnet.NOUN
    elif nltk_tag.startswith('R'):
        return wordnet.ADV
    else:
        return None

# 기존 텍스트/정제 텍스트 반환
def cleaning_documents(doc):
    sent_list = nltk.sent_tokenize(doc)
    temp_list = sent_list
    doc = re.sub(r"[^\w\s!?.\-']", "", doc)
    for i in range(len(temp_list)):
        if temp_list[i][0].isupper() and temp_list[i][1].islower():
            temp_list[i] = temp_list[i][0].lower() + temp_list[i][1:]

    temp_list = [sent[:-1] if sent.endswith('.') else sent for sent in temp_list]

    # 텍스트 정제 (불용어 제거)
    stop_words = set(stopwords.words('english'))
    clean_list = []
    for sent in temp_list:
        words = word_tokenize(sent)
        words = [word for word in words if word not in stop_words]
        words = ' '.join(words)
        clean_list.append(words)

    # 텍스트 정제 (형태소 분석)
    lemmatized_list = []
    for i in clean_list:
        pos_tagged = nltk.pos_tag(nltk.word_tokenize(i))
        wordnet_tagged = list(map(lambda x: (x[0], pos_tagger(x[1])), pos_tagged))
        lemmatized_sent = []
        for word, tag in wordnet_tagged:
            if tag is not None:
                lemmatized_sent.append(wnl.lemmatize(word, tag))
        lemmatized_sent = ' '.join(lemmatized_sent)
        lemmatized_list.append(lemmatized_sent)
    return lemmatized_list, sent_list

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [ ]:
c_temp = df['content'].apply(text_preprocessing)
temp = c_temp.apply(cleaning_documents)
cleaned = temp.apply(lambda x: x[0])
original = temp.apply(lambda x: x[1])

# TF-IDF + TEXTRANK 요약

In [ ]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import normalize

tfidf = TfidfVectorizer()
count_vec = CountVectorizer()
graph= []

def graph_by_sent(sent):
  tfidf_arr = tfidf.fit_transform(sent).toarray()
  graph = np.dot(tfidf_arr, tfidf_arr.T)
  return graph

def textrank(graph, d=0.85):
  A = graph
  matrix_size = A.shape[0]
  for i in range(matrix_size):
    A[i,i] = 0
    link_sum = np.sum(A[:,i])
    if link_sum != 0:
      A[:, i] /= link_sum
    A[:,i] *= -d
    A[i,i] = 1
  B = (1-d) * np.ones((matrix_size, 1))
  ranks = np.linalg.solve(A, B)
  return {idx: r[0] for idx, r in enumerate(ranks)}

def summarize(sorted_sent_rank, num_indices=5):
    extracted_sentences = []

    for idx, rank in enumerate(sorted_sent_rank):
        indices = rank[:num_indices]
        indices.sort()
        sentences = [original[idx][j] for j in indices]
        extracted_sentences.append(sentences)

    return pd.Series(extracted_sentences)

def keywords(sent):
  noun_list = []
  for i in sent:
       pos_tagged = nltk.pos_tag(nltk.word_tokenize(i))
       wordnet_tagged = list(map(lambda x: (x[0], pos_tagger(x[1])), pos_tagged))
       noun_temp = []
       for word, tag in wordnet_tagged:
        if tag == wordnet.NOUN:
          noun_temp.append(word)
       noun_temp = ' '.join(noun_temp)
       noun_list.append(noun_temp)

  tfidf_arr = tfidf.fit_transform(noun_list).toarray()
  vocab = tfidf.vocabulary_
  tfidf_mtx = pd.DataFrame(tfidf_arr, columns = vocab)
  return tfidf_mtx.sum().sort_values(ascending=False)[:5].to_dict()

sent_graph = cleaned.apply(graph_by_sent)
sent_rank = sent_graph.apply(textrank)
sorted_sent_rank = sent_rank.apply(lambda x: [k for k, _ in sorted(x.items(), key=lambda k: k[1], reverse=True)])
extracted_sentences = summarize(sorted_sent_rank)
keyword_rank = cleaned.apply(keywords)

summary = extracted_sentences.apply(lambda x: ' '.join(x))

### BART

In [ ]:
!pip install --upgrade tensorflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install --upgrade transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 45.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 24.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 96.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 25.4 MB/s eta 0:00:00


In [ ]:
from transformers import pipeline
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

In [ ]:
original = original.apply(lambda x: ' '.join(x))
original_reg = original.apply(lambda x: x[:1024] if len(x) > 1024 else x)
bart_summary = original_reg.apply(summarizer)
bart = bart_summary.apply(lambda x: x[0]['summary_text'])

Your max_length is set to 142, but your input_length is only 113. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=56)


# 최종 데이터프레임

In [ ]:
df['bart'] = bart
df['summary'] = summary
df['keyword'] = keyword_rank

In [ ]:
df

,title,time,stocks,content,source,link,bart,summary,keyword
0,Germany refusing Intel's additional demand for...,"Jun 11, 2023 12:29AM",INTC:-1.51%,(Reuters) - Germany's Finance Minister Christi...,By Reuters,https://www.investing.com/news/stock-market-ne...,German Finance Minister Christian Lindner refu...,germany's Finance Minister Christian Lindner i...,"{'budget': 1.0, 'lindner': 0.8108428248189414,..."
1,"Zelenskiy says counteroffensive actions ""takin...","Jun 10, 2023 09:20PM",,By Tom Balmforth KYIV (Reuters) -President Vol...,By Reuters,https://www.investing.com/news/world-news/zele...,Ukrainian president Volodymyr Zelenskiy says h...,president Volodymyr Zelenskiy acknowledged on ...,"{'ass': 1.5678693096710512, 'mood': 1.54777726..."
2,Energy & precious metals - weekly review and o...,"Jun 11, 2023 05:03AM","XAU/USD:-0.24%, Gold:-0.14%, LCO:-1.21%, CL:-1...",Investing.com -- “Patience” - it’s a commodity...,By Investing.com,https://www.investing.com/news/commodities-new...,The Fed is expected to vote for a break from a...,"meanwhile, at OPEC, Abdulaziz will require mor...","{'mobility': 4.281270639291539, 'intermediate'..."
3,Bidens offer 'joy' at White House Pride event ...,"Jun 10, 2023 04:15PM",,By Andrea Shalal WASHINGTON (Reuters) - U.S. P...,By Reuters,https://www.investing.com/news/world-news/bide...,U.S. President Joe Biden hosted the largest Pr...,U.S. President Joe Biden on Saturday hosted th...,"{'safety': 1.0352565863228482, 'measure': 1.0,..."
4,Four Colombian children found alive in jungle ...,"Jun 09, 2023 08:22PM",,By Luis Jaime Acosta BOGOTA (Reuters) - Four c...,By Reuters,https://www.investing.com/news/world-news/four...,Four children from an Indigenous community in ...,four children from an Indigenous community in ...,"{'baby': 1.544521150404306, 'petro': 1.2820649..."
5,Why is the crypto market down today?,"Jun 10, 2023 08:19AM","BTC/USD:+0.09%, USDT/USD:-0.03%, BTC/USD:+0.11...",The cryptocurrency market fell 7% below $1 tri...,By Cointelegraph,https://www.investing.com/news/cryptocurrency-...,The cryptocurrency market fell 7% below $1 tri...,the cryptocurrency market fell 7% below $1 tri...,"{'nasdaq': 0.7405893144386846, 'industry': 0.6..."
6,Twitter is refusing to pay its Google Cloud bi...,"Jun 10, 2023 07:22PM","GOOGL:+0.07%, AMZN:-0.66%, GOOG:+0.16%",(Reuters) - Twitter has refused to pay its Goo...,By Reuters,https://www.investing.com/news/stock-market-ne...,Twitter has refused to pay its Google (NASDAQ:...,twitter has refused to pay its Google (NASDAQ:...,"{'email': 1.3330235488395068, 'saturday': 1.11..."
7,U.S. gives Havana embassy a facelift after yea...,"Jun 10, 2023 07:04AM",,By Dave Sherwood HAVANA (Reuters) -When the U....,By Reuters,https://www.investing.com/news/world-news/us-g...,The U.S. embassy in Havana reopened last May t...,when the U.S. embassy in Havana reopened last ...,"{'granite': 2.1958512773026935, 'state': 2.094..."
8,Trump magnifies attacks on Justice Department ...,"Jun 10, 2023 04:18PM",,By Nathan Layne (Reuters) -Former President Do...,By Reuters,https://www.investing.com/news/world-news/trum...,Former President Donald Trump accused the Just...,former President Donald Trump used the first p...,"{'york': 3.3422937712224354, 'speak': 2.322453..."
9,BioNTech faces first German lawsuit over alleg...,"Jun 11, 2023 05:56AM",PFE:-0.31%,By Ludwig Burger and Patricia Weiss HAMBURG (R...,By Reuters,https://www.investing.com/news/stock-market-ne...,bioNTech will go to court on Monday to defend...,bioNTech will go to court on Monday to defend ...,"{'client': 1.9304926822314872, 'law': 1.916043..."


# 번역 처리 (googletrans)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os

from google.api_core.exceptions import InvalidArgument

from google.cloud import translate_v2

credential_path = "/content/drive/MyDrive/sacred-footing-389413-b5fa42c20a9b.json"

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = credential_path

translate_client = translate_v2.Client()

In [ ]:
bart_kor = bart.apply(lambda x: translate_client.translate(x, target_language="ko"))
summary_kor = summary.apply(lambda x: translate_client.translate(x, target_language="ko"))
bart_ko = bart_kor.apply(lambda x: x['translatedText'])
summary_ko = summary_kor.apply(lambda x: x['translatedText'])
df['bart_ko'] = bart_ko
df['summary_ko'] = summary_ko

In [ ]:
title_kor = df['title'].apply(lambda x: translate_client.translate(x, target_language="ko"))
title_ko = title_kor.apply(lambda x: x['translatedText'])
df['title_ko'] = title_ko

In [ ]:
df

,title,time,stocks,content,source,link,bart,summary,keyword,bart_ko,summary_ko,title_ko
0,Germany refusing Intel's additional demand for...,"Jun 11, 2023 12:29AM",INTC:-1.51%,(Reuters) - Germany's Finance Minister Christi...,By Reuters,https://www.investing.com/news/stock-market-ne...,German Finance Minister Christian Lindner refu...,germany's Finance Minister Christian Lindner i...,"{'budget': 1.0, 'lindner': 0.8108428248189414,...",크리스티안 린드너(Christian Lindner) 독일 재무장관은 170억 유로(...,독일의 재무장관 크리스티안 린드너(Christian Lindner)는 170억 유로...,"독일, 칩 공장에 대한 인텔의 추가 보조금 요구 거부 - FT"
1,"Zelenskiy says counteroffensive actions ""takin...","Jun 10, 2023 09:20PM",,By Tom Balmforth KYIV (Reuters) -President Vol...,By Reuters,https://www.investing.com/news/world-news/zele...,Ukrainian president Volodymyr Zelenskiy says h...,president Volodymyr Zelenskiy acknowledged on ...,"{'ass': 1.5678693096710512, 'mood': 1.54777726...",볼로디미르 젤렌스키 우크라이나 대통령은 자신의 군대가 &quot;반격 및 방어 작전...,볼로디미르 젤렌스키 대통령은 토요일 크렘린 지도자 블라디미르 푸틴이 키예프가 오랫동...,Zelenskiy는 우크라이나에서 반격 조치가 &quot;일어나고 있다&quot;고...
2,Energy & precious metals - weekly review and o...,"Jun 11, 2023 05:03AM","XAU/USD:-0.24%, Gold:-0.14%, LCO:-1.21%, CL:-1...",Investing.com -- “Patience” - it’s a commodity...,By Investing.com,https://www.investing.com/news/commodities-new...,The Fed is expected to vote for a break from a...,"meanwhile, at OPEC, Abdulaziz will require mor...","{'mobility': 4.281270639291539, 'intermediate'...",연준은 15개월 전에 시작된 금리 인상 캠페인 중단에 투표할 것으로 예상됩니다. 중...,한편 OPEC에서 Abdulaziz는 사우디의 감산이 널리 알려졌음에도 불구하고 석...,에너지 및 귀금속 - 주간 검토 및 전망
3,Bidens offer 'joy' at White House Pride event ...,"Jun 10, 2023 04:15PM",,By Andrea Shalal WASHINGTON (Reuters) - U.S. P...,By Reuters,https://www.investing.com/news/world-news/bide...,U.S. President Joe Biden hosted the largest Pr...,U.S. President Joe Biden on Saturday hosted th...,"{'safety': 1.0352565863228482, 'measure': 1.0,...",조 바이든 미국 대통령은 토요일 백악관 역사상 가장 큰 프라이드 행사를 주최했습니다...,조 바이든 미국 대통령은 토요일 백악관 역사상 가장 큰 프라이드 축하 행사를 주최하...,Bidens는 LGBTQ 공격이 증가함에 따라 백악관 프라이드 행사에서 &#39;기...
4,Four Colombian children found alive in jungle ...,"Jun 09, 2023 08:22PM",,By Luis Jaime Acosta BOGOTA (Reuters) - Four c...,By Reuters,https://www.investing.com/news/world-news/four...,Four children from an Indigenous community in ...,four children from an Indigenous community in ...,"{'baby': 1.544521150404306, 'petro': 1.2820649...",콜롬비아의 한 원주민 공동체에서 온 네 명의 어린이들이 그들이 여행하던 비행기가 추...,콜롬비아의 구스타보 페트로 콜롬비아 대통령은 그들이 타고 있던 비행기가 울창한 정글...,"콜롬비아 어린이 4명, 비행기 추락 몇 주 만에 정글서 숨진 채 발견"
5,Why is the crypto market down today?,"Jun 10, 2023 08:19AM","BTC/USD:+0.09%, USDT/USD:-0.03%, BTC/USD:+0.11...",The cryptocurrency market fell 7% below $1 tri...,By Cointelegraph,https://www.investing.com/news/cryptocurrency-...,The cryptocurrency market fell 7% below $1 tri...,the cryptocurrency market fell 7% below $1 tri...,"{'nasdaq': 0.7405893144386846, 'industry': 0.6...","암호화폐 시장은 6월 10일 1조 달러 미만으로 7% 하락했으며, 업계 최대 거래소...","암호화폐 시장은 6월 10일 1조 달러 미만으로 7% 하락했으며, 업계 최대 거래소...",오늘 암호화폐 시장이 하락한 이유는 무엇입니까?
6,Twitter is refusing to pay its Google Cloud bi...,"Jun 10, 2023 07:22PM","GOOGL:+0.07%, AMZN:-0.66%, GOOG:+0.16%",(Reuters) - Twitter has refused to pay its Goo...,By Reuters,https://www.investing.com/news/stock-market-ne...,Twitter has refused to pay its Google (NASDAQ:...,twitter has refused to pay its Google (NASDAQ:...,"{'email': 1.3330235488395068, 'saturday': 1.11...",트위터는 이번 달 계약 갱신이 다가옴에 따라 Google(NASDAQ:GOOGL) ...,트위터는 이번 달 계약 갱신을 앞두고 구글(NASDAQ:GOOGL) 클라우드 청구서...,Twitter는 Google Cloud 청구서 지불을 거부합니다.
7,U.S. gives Havana embassy a facelift after yea...,"Jun 10, 2023 07:04AM",,By Dave Sherwood HAVANA (Reuters) -When the U....,By Reuters,https://www.investing.com/news/world-news/us-g...,The U.S. embassy in Havana reopened last May t...,when the U.S. embassy in Havana reopened last ...,"{'granite': 2.1958512773026935, 'state': 2.094...",아바나 주재 미국 대사관은 지난 5월 거의 5년 간의 공백 끝에 비자를 찾는 쿠바인...,지난 5월 아바나 주재 미국 대사관이 거의 

In [ ]:
df.to_csv('final_datafile.csv')

# PDF 생성

In [ ]:
!pip install fpdf

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for fpdf: filename=fpdf-1.7.2-py2.py3-none-any.whl size=40704 sha256=6adff41c313b91ed7941893c6379c6d2940da4520e16ec0336dbbe1ccb66f786
  Stored in directory: /root/.cache/pip/wheels/f9/95/ba/f418094659025eb9611f17cbcaf2334236bf39a0c3453ea455
Successfully built fpdf


In [ ]:
from fpdf import FPDF

ch = 4

class PDF(FPDF):
    def __init__(self):
        super().__init__()

    def footer(self):
        self.set_y(-15)
        self.set_font('Arial', '', 12)
        self.cell(0, 10, f'Page{self.page_no()}', 0, 0, 'C')

pdf = PDF()

for i in range(len(df)):
  pdf.add_page()
  pdf.image('/content/drive/MyDrive/letterhead.png', 0, 0, 210)

  pdf.add_font('eunjin', '', '/content/drive/MyDrive/font/Eunjin.ttf', uni=True)
  pdf.set_font('eunjin', '', 10)
  pdf.cell(w=0, h=100, txt = df['title_ko'][i], ln = 1)

  pdf.set_font('Arial','', 7)
  pdf.cell(w=30, h=ch, txt="Date: ", ln=0)
  pdf.cell(w=30, h=ch, txt= df['time'][i], ln=1)
  pdf.cell(w=30, h=ch, txt="Source: ", ln=0)
  pdf.cell(w=30, h=ch, txt= df['source'][i], ln=1)

  pdf.set_font('Arial','', 8)
  pdf.cell(w=30, h=ch, txt="Stocks: ", ln=0)
  pdf.cell(w=30, h=ch, txt= df['stocks'][i], ln=1)

  pdf.set_font('eunjin', '', 8)
  pdf.cell(w=0, h=20, txt="바트 요약: ", ln=1)
  pdf.ln(1)
  pdf.multi_cell(w=0, h=5, txt= df['bart_ko'][i])
  pdf.ln(1)
  pdf.cell(w=0, h=25, txt="텍스트랭크 요약: ", ln=1)
  pdf.ln(1)
  pdf.multi_cell(w=0, h=5, txt = df['summary_ko'][i])
  pdf.ln(3)

  pdf.set_font('eunjin','', 8)
  pdf.cell(w=30, h=ch, txt="키워드: ", ln=0)
  pdf.set_font('Arial','', 12)
  pdf.cell(w=30, h=ch, txt= str(list(df['keyword'][i].keys())), ln=1)
  pdf.ln(2)

  pdf.set_font('eunjin','', 8)
  pdf.cell(w=30, h=ch, txt="링크: ", ln=0)
  pdf.set_font('Arial','', 8)
  pdf.cell(w=30, h=ch, txt= df['link'][i])

pdf.output(f'./news_summarize.pdf', 'F')

/usr/local/lib/python3.10/dist-packages/fpdf/ttfonts.py:831: UserWarning: missing glyph 11741
  warnings.warn("missing glyph %s" % (originalGlyphIdx))
/usr/local/lib/python3.10/dist-packages/fpdf/ttfonts.py:670: UserWarning: cmap value too big/small: -43956
  warnings.warn("cmap value too big/small: %s" % cm)
/usr/local/lib/python3.10/dist-packages/fpdf/ttfonts.py:670: UserWarning: cmap value too big/small: -43958
  warnings.warn("cmap value too big/small: %s" % cm)
/usr/local/lib/python3.10/dist-packages/fpdf/ttfonts.py:670: UserWarning: cmap value too big/small: -43961
  warnings.warn("cmap value too big/small: %s" % cm)
/usr/local/lib/python3.10/dist-packages/fpdf/ttfonts.py:670: UserWarning: cmap value too big/small: -43968
  warnings.warn("cmap value too big/small: %s" % cm)
/usr/local/lib/python3.10/dist-packages/fpdf/ttfonts.py:670: UserWarning: cmap value too big/small: -43971
  warnings.warn("cmap value too big/small: %s" % cm)
/usr/local/lib/python3.10/dist-packages/fpdf/ttfo

''

In [1]:
import os
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.encoders import encode_base64
from email.mime.base import MIMEBase

my_mail = "emerald679@naver.com"
pwd = input("비밀번호: ")
to_mail = "emerald679@naver.com"

# 메일 제목/ 받는 사람/ 보내는 사람 정보
msg = MIMEMultipart()
msg['Subject'] = "NEWS SUMMARIZER"
msg['From'] = my_mail
msg['To'] = to_mail

# 본문 내용
text = MIMEText("오늘자 Investing.com 요약본입니다")
msg.attach(text)

files = list()
files.append('/content/news_summarize.pdf')

for f in files:
    part = MIMEBase('application', "octet-stream")
    part.set_payload(open(f, "rb").read())
    encode_base64(part)
    part.add_header('Content-Disposition', 'attachment; filename="%s"' % os.path.basename(f))
    msg.attach(part)

# 이메일 전송
smtp = smtplib.SMTP("smtp.naver.com", 587)
smtp.starttls()
smtp.login(user=my_mail, password=pwd)
smtp.sendmail(my_mail, to_mail, msg.as_string())
smtp.close()

KeyboardInterrupt: ignored